In [1]:
#given .log file:
#     create datframe of points
#     (array i = source i + 1)
#     dataframe: key:timestamp -> 2D array[[id, tag, x, y,z, activity]
#                                    [id, tag, x, y,z, activity]
#                                    [id, tag, x, y,z, activity]
#                                    [id, tag, x, y,z, activity]]
#     in dataframe:
#     if(id of all source == 0):
#         delete datapoint
#     else
#         convert timestamp to time
#  
#     return dataframe

In [2]:
#merge log files with same starting time

In [3]:
import pandas as pd
import re 
import json
import datetime
import os
import time
import glob

In [4]:
#todo
#import time extract and duration milliseconds to make it cleaner(maybe put these functions in different files so all
#files can extract them)
#add mircophone number to dataframe
#fix time(it not showing correct dates)

#in a new file write script to create dataframes for all files, and merge them into 1 dataframe and sort it by time

In [46]:
def timeExtract(filename):
    with open(filename, 'rb') as f:
        # Start counting from the last byte
        counter = 1
        # Go to the 2nd byte before the end of the last line
        f.seek(-2, 2) 
        while f.read(1) != b'\n':
            f.seek(-2, 1)
            counter=counter+1
        endTime_line = f.readline().decode()
        # Go to the 2nd byte before the end of the last second line
        f.seek(-counter-2, 2)
        while f.read(1) != b'\n':
            f.seek(-2, 1)
        startTime_line = f.readline().decode()

    return [startTime_line, endTime_line]


def durationinMicroseconds(filename):
    startTime = timeExtract(filename)[0].split()[2:]
    endTime = timeExtract(filename)[1].split()[2:]
    startTimeStr = startTime[0] + ' ' + startTime[1]
    endTimeStr = endTime[0] + ' ' + endTime[1]
    T1 = datetime.datetime.strptime(startTimeStr, '%Y-%m-%d %H:%M:%S.%f')
    T2 = datetime.datetime.strptime(endTimeStr, '%Y-%m-%d %H:%M:%S.%f')
    delta = T2-T1
    duration = delta.seconds*1000000 + delta.microseconds
    
    return duration, T1, T2



def extractDirectionalities(filename):
    with open(filename, 'r') as f:
        text = f.read()
        # Use repex to store blocks of data into a list
    data = re.split('(?<=})\n(?={)', text) 
        # Delete the time info from the last data block
    tmp = data[-1][:(data[-1].rfind("}")+1)]
    data[-1] = tmp
        
    #list of src blocks 
    srcList = [json.loads(block)["src"] for block in data]
    
    #initialize dataframe to have colums: timestamp, time, data inside source
    #timestamp is the initial time stamp
    #time is the datetime value converted from the timestamp and intitial time
    #source is a 4 by 6 array where the rows are the source, and the columns are the source values
    df = pd.DataFrame(columns = ['timestamp', 'time', 'id', 'x', 'y', 'z', 'activity'])
    
    #Used for calculating timestamps -> time
    duration, startTime, endTime = durationinMicroseconds(filename)
    t = duration/len(data)
    
    
    index = 1
    for block in srcList:
        if block[0]["id"] != 0 or block[1]["id"] != 0 or block[2]["id"] != 0 or block[3]["id"] != 0:
            timesince = datetime.datetime.now() - startTime
            time = timesince.total_seconds() + (index - 1.0) * t
            for i in range(0, 4):
                if block[i]['id'] != 0:
                    df = df.append(pd.DataFrame({"timestamp": [index], "time":datetime.datetime.fromtimestamp(time).strftime("%A, %B %d, %Y %I:%M:%S"), "id": block[i]["id"], "x": block[i]["x"], "y": block[i]["y"], "z": block[i]["z"], "activity": block[i]["activity"]}, index=[0]))
        index = index + 1
            
    return(df)
            
            
        
        
            

In [47]:
filename = "/Users/ardelalegre/CSE4223-ODAS/cSST_2019-09-09_14_55_03.log"
dataframe = extractDirectionalities(filename)


In [49]:
dataframe.head()

,timestamp,time,id,x,y,z,activity
0,300,"Saturday, April 25, 1970 10:02:26",2,0.701,-0.561,0.441,1.000
0,301,"Saturday, April 25, 1970 12:15:45",2,0.701,-0.561,0.441,0.999
0,302,"Saturday, April 25, 1970 02:29:03",2,0.701,-0.561,0.441,0.999
0,303,"Saturday, April 25, 1970 04:42:22",2,0.701,-0.561,0.441,0.999
0,304,"Saturday, April 25, 1970 06:55:40",2,0.701,-0.561,0.441,0.998
